Careless responder feature engineering and analysis

In [ ]:
#getting and working with data
import pandas as pd
import numpy as np
import re
import os
from itertools import groupby
import datetime as dt
import scipy as sp

#visualizing results
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
#import yellowbrick as yb

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import warnings; warnings.simplefilter('ignore')
np.set_printoptions(suppress=True)

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics.cluster import silhouette_score

In [ ]:
data_path = 'C:/Users/Schindler/Documents/Schindler_Lab/Data/Clinical projects/TILES/final_data/final_data_complete.pkl'

In [ ]:
#read in csv containing data from all surveys
data = pd.read_pickle(data_path)
data = pd.DataFrame(data = data)
data.reset_index(inplace=True, drop=True)

print('Original data shape:\n', data.shape, '\n')
#ensure no replicate ID (212 participants in study)
print('Original data unique IDs:\n', data['ParticipantID'].unique().shape, '\n')
#ensure no replicate ID (212 participants in study)
print('Original data unique IDs:\n', data['MitreID'].unique().shape, '\n')
#how much missing data is there?
print('Original data missing value counts:\n', data.isnull().sum(), '\n')
#what is the data type of each column?
print('Original data data types:\n', data.info(), '\n')

In [ ]:
data['survey_type'].value_counts()

In [ ]:
#should be 71
len(data['wave_study_day'].unique())

In [ ]:
#create study date bins
data['wave_study_date_bin'] = pd.cut(data['wave_study_day'], 5)
data['wave_study_date_bin'].value_counts()

In [ ]:
data.head()

## Features and clustering on Engage surveys

### notes for CR features for Engage surveys

Context question
- Semantic Antonyms
    - if context1 = home (0), then context2 ≠ work and work related (0)
    - if context1 = work (1), then context2 ≠ leisure sports (4), household activities (7), org/civic (11)
- Semantic Synonyms
    - if context1 = work (1), then context2 most likely work and work related (0)
    - If context1 = vehicle (4), then context2 most likely travel or commute (12)
- Internal consistency
    - if context1 = 5 (other) then should have a write in
    - if context2 = 13 (other) then should have a write in

Longstring
- All questions use same scale (1=not at all, 7=very much), but there are 5 different constructs assessed

Semantic consistency
- Internal consistency (within construct) should be greater than consistency across constructs

Semantic synonyms 
- not applicable 

Semantic antonyms
- Hindrance stressors should be negatively correlated with support 


In [ ]:
#split off completed engage and related columns
engage_only = data[(data['survey_type'] == 'engage_psycap') & (data['completed'] == 1.0)]

print(engage_only.shape)
engage_only['ParticipantID'].unique().shape

In [ ]:
#context related CR features
        
context_homevsworking = []
context_workvsactivities = []
context_workvswork = []
context_drivevsdrive = []
write_in_location = []
write_in_activity = []
hinderance_vs_support = []

for index, row in engage_only.iterrows():
    
    #if at home should not be working
    if (row['location_num'] == 0) & (row['activity_num'] == 0):
        context_homevsworking.append(1)
    else:
        context_homevsworking.append(0)
        
    #if at work should not be playing sports, household activities, civic duties
    if (row['location_num'] == 1) & ((row['activity_num'] == 4) | (row['activity_num'] == 7) | (row['activity_num'] == 11)):
        context_workvsactivities.append(1)
    else:
        context_workvsactivities.append(0)
    
    #if at work should be working
    if (row['location_num'] == 1) & (row['activity_num'] != 0):
        context_workvswork.append(1)
    else:
        context_workvswork.append(0)
        
    #if at vehicle should be driving/travel    
    if (row['location_num'] == 4) & (row['activity_num'] != 12):
        context_drivevsdrive.append(1)
    else:
        context_drivevsdrive.append(0)
    
    #if put other then should have write in
    if row['location_num'] == 5:
        write_in_location.append(1)
    else:
        write_in_location.append(0)
        
    if (row['activity_num'] == 13):
        write_in_activity.append(1)
    else:
        write_in_activity.append(0)
        
    #Hindrance stressors should be negatively correlated with support
    num = np.std([row['support_mgt'], row['hindrance_mgt']])
    hinderance_vs_support.append(abs(num))

#context checks
engage_only['context_homevsworking'] = context_homevsworking
engage_only['context_workvsactivities'] = context_workvsactivities
engage_only['context_workvswork'] = context_workvswork
engage_only['context_drivevsdrive'] = context_drivevsdrive

engage_only['write_in_location'] = write_in_location
engage_only['write_in_activity'] = write_in_activity
engage_only['hinderance_vs_support'] = hinderance_vs_support

In [ ]:
#long string analysis (e.g. max length of same number answered for engage_3:engage_29)
#create features related to long string analysis (feature of how long the string is and feature of what the string consisted of)

max_strings = []
max_answers = []

for index, row in engage_only.iterrows():
    
    groups = groupby(row['engage_3':'engage_29'])
    result = [(label, sum(1 for _ in group)) for label, group in groups]

    max_pair = max(result, key=lambda x:x[1])
    max_string_length = max_pair[1]
    max_answer = max_pair[0]

    max_strings.append(max_string_length)
    
    max_answers.append(max_answer)
    
engage_only['longest_string_count'] = max_strings
engage_only['longest_string_answer'] = max_answers

In [ ]:
#seeded skew and kurtosis (trying to deal with 0 skew of all same answer vs normally distributed answers)
std_seeded = []
skew_seeded = []
kurt_seeded = []

for index, row in engage_only.iterrows():
    num_std = np.std(np.append(row.loc['engage_3':'engage_29'].dropna().values, 0.0))
    std_seeded.append(num_std)
    num_skew = sp.stats.skew(np.append(row.loc['engage_3':'engage_29'].dropna().values, 0.0))
    skew_seeded.append(num_skew)
    num_kurt = sp.stats.kurtosis(np.append(row.loc['engage_3':'engage_29'].dropna().values, 0.0))
    kurt_seeded.append(num_kurt)
    
engage_only['std_seeded'] = std_seeded    
engage_only['skew_seeded'] = skew_seeded
engage_only['kurt_seeded'] = kurt_seeded

In [ ]:
#create feature that is surevy response skew
engage_only['std'] = engage_only.loc[:, 'engage_3':'engage_29'].std(axis=1)
engage_only['skew'] = engage_only.loc[:, 'engage_3':'engage_29'].skew(axis=1)
engage_only['kurtosis'] = engage_only.loc[:, 'engage_3':'engage_29'].kurtosis(axis=1)

In [ ]:
engage_only.dropna(subset=['hinderance_vs_support'], inplace=True)
engage_only.shape

In [ ]:
#viz relationship and correlation across possible features
potential_features = ['longest_string_count', 'longest_string_answer', 'std_seeded', 'skew_seeded', 'kurt_seeded', 
                      'std', 'skew', 'kurtosis', 'time_to_complete', 'hinderance_vs_support', 
                      'context_homevsworking', 'context_workvsactivities', 'context_workvswork', 'context_drivevsdrive', 
                      'write_in_location', 'write_in_activity']
potential_features_cont = ['hinderance_vs_support', 'longest_string_count', 'longest_string_answer', 
                           'std_seeded', 'skew_seeded', 'kurt_seeded', 'std', 'skew', 'kurtosis', 'time_to_complete']
potential_features_binary = ['context_homevsworking', 'context_workvsactivities', 'context_workvswork', 
                             'context_drivevsdrive', 'write_in_location', 'write_in_activity']

engage_only_features_potential = engage_only[potential_features]
engage_only_features_cont = engage_only[potential_features_cont]
engage_only_features_potential.corr()

In [ ]:
for feature in potential_features_cont:
    sns.jointplot(engage_only_features_potential['longest_string_count'], engage_only_features_potential[feature], kind='hex')
    plt.show()

In [ ]:
for feature in potential_features_binary:
    print(engage_only[feature].value_counts())
    sns.countplot(x='longest_string_count', hue=feature, data=engage_only_features_potential)
    plt.show()

In [ ]:
sns.pairplot(engage_only[potential_features_cont], diag_kind='kde')

In [ ]:
engage_only_features = engage_only[['MitreID', 'hinderance_vs_support', 'longest_string_count', 'skew_seeded', 'kurt_seeded']]
engage_only_features.set_index('MitreID', inplace=True)
engage_only_features.head()

In [ ]:
# center and scale the data
scaler = StandardScaler()

engage_survey_features_scaled = scaler.fit_transform(engage_only_features)

In [ ]:
k_range = range(2,10)
scores = []
for k in k_range:
    km_ss = KMeans(n_clusters=k, random_state=1)
    km_ss.fit(engage_survey_features_scaled)
    scores.append(silhouette_score(engage_survey_features_scaled, km_ss.labels_))

# plot the results
plt.plot(k_range, scores)
plt.xlabel('Number of clusters')
plt.ylabel('Silhouette Coefficient')
plt.title('PF kmeans at survey level')

In [ ]:
engage_km_survey = KMeans(n_clusters=2,random_state=1234)
engage_km_survey.fit(engage_survey_features_scaled)
print(silhouette_score(engage_survey_features_scaled, engage_km_survey.labels_))

engage_only_features['kmeans_scaled_survey'] = [label for label in engage_km_survey.labels_ ]
engage_only_features_cont['kmeans_scaled_survey'] = [label for label in engage_km_survey.labels_ ]
engage_only['kmeans_scaled_survey'] = [label for label in engage_km_survey.labels_ ]

engage_only['kmeans_scaled_survey'].value_counts()

In [ ]:
sns.pairplot(engage_only_features_cont, hue = 'kmeans_scaled_survey', diag_kind='kde')

In [ ]:
for feature in potential_features:
    sns.barplot(x='kmeans_scaled_survey', y=feature, data=engage_only)
    plt.show()

In [ ]:
for feature in potential_features:
    sns.barplot(x='longest_string_count', y=feature, hue='kmeans_scaled_survey', data=engage_only)
    plt.show()

In [ ]:
for feature in potential_features_binary:
    print(engage_only.groupby("kmeans_scaled_survey")[feature].value_counts())
    sns.countplot(x='longest_string_count', hue=feature, data=engage_only_features_potential)
    plt.show()

In [ ]:
engage_only.groupby(['MitreID', 'kmeans_scaled_survey'])['ID'].count()

In [ ]:
#add cluster 1 % to final data
participants = data['MitreID'].unique()

for part in participants:
    if part in engage_only_features.index:
        perc = engage_only_features[(engage_only_features.index == part) & (engage_only_features['kmeans_scaled_survey'] == 1)].shape[0] / engage_only_features[engage_only_features.index == part].shape[0]

        data.loc[data['MitreID'] == part, 'engage_CR_perc'] = perc
        engage_only.loc[engage_only['MitreID'] == part, 'engage_CR_perc'] = perc
    
    else:
        data.loc[data['MitreID'] == part, 'engage_CR_perc'] = np.nan
        engage_only.loc[engage_only['MitreID'] == part, 'engage_CR_perc'] = np.nan

In [ ]:
sns.countplot(x='wave_study_date_bin', hue='kmeans_scaled_survey', data=engage_only)

In [ ]:
sns.barplot(x='MitreID', y='engage_CR_perc', data=engage_only)

In [ ]:
participants = engage_only['MitreID'].unique()

for part in participants:
    data_part = engage_only[engage_only['MitreID'] == part]
    sns.countplot(x='wave_study_day', hue='kmeans_scaled_survey', data=data_part)
    plt.show()

## Features and clustering on PF surveys

### notes for CR features for Psych Flex

Should have answered every question

Longstring
- Legitimate longstrings of  ≥ 8 are unlikely for response “5”
    - make column with longest string
    - make column with number that longest string consisted of

Semantic consistency
- Legitimate scores of pf_mgt=5 are almost impossible

Semantic antonyms
- Not applicable

Semantic synonyms 
- not applicable 


In [ ]:
#split off completed PF and related columns
psych_flex_only = data[(data['survey_type'] == 'psych_flex') & (data['completed'] == 1.0)]

print(psych_flex_only.shape)
psych_flex_only['ParticipantID'].unique().shape

In [ ]:
#long string analysis (e.g. max length of same number answered for pf_03:pf_15)
#create features related to long string analysis (feature of how long the string is and feature of what the string consisted of)

max_strings = []
max_answers = []

for index, row in psych_flex_only.iterrows():
    
    groups = groupby(row['pf_03':'pf_15'])
    result = [(label, sum(1 for _ in group)) for label, group in groups]

    max_pair = max(result, key=lambda x:x[1])
    max_string_length = max_pair[1]
    max_answer = max_pair[0]

    max_strings.append(max_string_length)
    
    max_answers.append(max_answer)
    
psych_flex_only['longest_string_count'] = max_strings
psych_flex_only['longest_string_answer'] = max_answers

In [ ]:
#seeded skew and kurtosis (trying to deal with 0 skew of all same answer vs normally distributed answers)
std_seeded = []
skew_seeded = []
kurt_seeded = []

for index, row in psych_flex_only.iterrows():
    
    try:
        num_std = np.std(np.append(row.loc['pf_03':'pf_15'].dropna().values, 0.0))
        std_seeded.append(num_std)
    except:
        std_seeded.append(np.nan)
        
    try:
        num_skew = sp.stats.skew(np.append(row.loc['pf_03':'pf_15'].dropna().values, 0.0))
        skew_seeded.append(num_skew)
    except:
        skew_seeded.append(np.nan)
    
    try:
        num_kurt = sp.stats.kurtosis(np.append(row.loc['pf_03':'pf_15'].dropna().values, 0.0))
        kurt_seeded.append(num_kurt)
    except:
        kurt_seeded.append(np.nan)
    
psych_flex_only['std_seeded'] = std_seeded    
psych_flex_only['skew_seeded'] = skew_seeded
psych_flex_only['kurt_seeded'] = kurt_seeded

In [ ]:
#create features without seed
psych_flex_only['std'] = psych_flex_only.loc[:, 'pf_03':'pf_15'].std(axis=1)
psych_flex_only['skew'] = psych_flex_only.loc[:, 'pf_03':'pf_15'].skew(axis=1)
psych_flex_only['kurtosis'] = psych_flex_only.loc[:, 'pf_03':'pf_15'].kurtosis(axis=1)

In [ ]:
psych_flex_only.dropna(subset=['kurtosis', 'skew_seeded'], inplace=True)
psych_flex_only.shape

In [ ]:
#viz relationship and correlation across possible features
potential_features = ['pf_mgt', 'longest_string_count', 'longest_string_answer', 'std_seeded', 'skew_seeded', 'kurt_seeded', 
                      'std', 'skew', 'kurtosis', 'time_to_complete', 'exp_neg', 'exp_pos', 'exp_neut']

psych_flex_only_features_potential = psych_flex_only[potential_features]
psych_flex_only_features_potential.corr()

In [ ]:
for feature in potential_features:
    sns.jointplot(psych_flex_only_features_potential['longest_string_count'], psych_flex_only_features_potential[feature], kind='hex')
    plt.show()

In [ ]:
sns.pairplot(psych_flex_only_features_potential, diag_kind='kde')

In [ ]:
psych_flex_only_features = psych_flex_only[['MitreID', 'longest_string_count', 'skew_seeded', 'kurt_seeded']]
psych_flex_only_features.set_index('MitreID', inplace=True)
psych_flex_only_features.head()

In [ ]:
# center and scale the data
scaler = StandardScaler()

PF_survey_features_scaled = scaler.fit_transform(psych_flex_only_features)

In [ ]:
k_range = range(2,10)
scores = []
for k in k_range:
    km_ss = KMeans(n_clusters=k, random_state=1)
    km_ss.fit(PF_survey_features_scaled)
    scores.append(silhouette_score(PF_survey_features_scaled, km_ss.labels_))

# plot the results
plt.plot(k_range, scores)
plt.xlabel('Number of clusters')
plt.ylabel('Silhouette Coefficient')
plt.title('PF kmeans at survey level')

In [ ]:
PF_km_survey = KMeans(n_clusters=2,random_state=1234)
PF_km_survey.fit(PF_survey_features_scaled)
print(silhouette_score(PF_survey_features_scaled, PF_km_survey.labels_))

psych_flex_only_features_potential['kmeans_scaled_survey'] = [label for label in PF_km_survey.labels_ ]
psych_flex_only_features['kmeans_scaled_survey'] = [label for label in PF_km_survey.labels_ ]
psych_flex_only['kmeans_scaled_survey'] = [label for label in PF_km_survey.labels_ ]

psych_flex_only['kmeans_scaled_survey'].value_counts()

In [ ]:
for feature in potential_features:
    sns.barplot(x='kmeans_scaled_survey', y=feature, data=psych_flex_only)
    plt.show()

In [ ]:
sns.pairplot(psych_flex_only_features_potential, hue = 'kmeans_scaled_survey', diag_kind='kde')

In [ ]:
#add cluster 1 % to final data
participants = data['MitreID'].unique()

for part in participants:
    if part in psych_flex_only_features.index:
        perc = psych_flex_only_features[(psych_flex_only_features.index == part) & (psych_flex_only_features['kmeans_scaled_survey'] == 0)].shape[0] / psych_flex_only_features[psych_flex_only_features.index == part].shape[0]

        data.loc[data['MitreID'] == part, 'pf_CR_perc'] = perc
        psych_flex_only.loc[psych_flex_only['MitreID'] == part, 'pf_CR_perc'] = perc
    
    else:
        data.loc[data['MitreID'] == part, 'pf_CR_perc'] = np.nan
        psych_flex_only.loc[psych_flex_only['MitreID'] == part, 'pf_CR_perc'] = np.nan

In [ ]:
sns.countplot(x='wave_study_date_bin', hue='kmeans_scaled_survey', data=psych_flex_only)

In [ ]:
sns.barplot(x='MitreID', y='pf_CR_perc', data=psych_flex_only)

In [ ]:
sns.jointplot(x='pf_CR_perc', y='engage_CR_perc', data=data, kind='reg')

In [ ]:
participants = psych_flex_only['MitreID'].unique()

for part in participants:
    print(part)
    data_part = psych_flex_only[psych_flex_only['MitreID'] == part]
    sns.countplot(x='wave_study_day', hue='kmeans_scaled_survey', data=data_part)
    plt.show()
    data_part = engage_only[engage_only['MitreID'] == part]
    sns.countplot(x='wave_study_day', hue='kmeans_scaled_survey', data=data_part)
    plt.show()

## Features and clustering on Mitre surveys: Jobs

### notes for CR features for Jobs

Context question (context 2 = activity, context 3 = location)
- Semantic Antonyms
    - if context3 = home (1), then context2 ≠ work and work related (1)
    - if context3 = work (2), then context2 ≠ leisure sports (3), household activities (6), org/civic (10)
- Semantic Synonyms
    - if context3 = work (2), then context2 most likely work and work related (1)
    - If context3 = vehicle (5), then context2 most likely travel or commute (11)

Affect/Anxiety/Stress
- Longstrings
    - All questions use same scale
- Semantic antonyms
    - Positive block (pan1-5) should be negatively correlated with negative block (pan6-10)

Task Perfomrance
- Longstrings
    - IRB questions use same scale
    - dalal questions use same scale
- Consistency
    - irb2, irb3, irb4 should be negatively correlated with irb6 and irb7
    - itp1, itp2, itp3 should be negatively correlated with irb6 and irb7
    - dalal1-8 should be negatively correlated with dalal9-dalal16



In [ ]:
#context related CR features
        
context_homevsworking = []
context_workvsactivities = []
context_workvswork = []
context_drivevsdrive = []

affect_check = []

for index, row in job_only.iterrows():
    
    #if at home should not be working
    if (row['context3'] == 1) & (row['context2'] == 1):
        context_homevsworking.append(1)
    else:
        context_homevsworking.append(0)
        
    #if at work should not be playing sports, household activities, civic duties
    if (row['context3'] == 2) & ((row['context2'] == 3) | (row['activity_num'] == 6) | (row['activity_num'] == 10)):
        context_workvsactivities.append(1)
    else:
        context_workvsactivities.append(0)
    
    #if at work should be working
    if (row['context3'] == 2) & (row['context2'] != 1):
        context_workvswork.append(1)
    else:
        context_workvswork.append(0)
        
    #if at vehicle should be driving/travel    
    if (row['context3'] == 5) & (row['context2'] != 11):
        context_drivevsdrive.append(1)
    else:
        context_drivevsdrive.append(0)
    

        
    #Positive block (pan1-5) should be negatively correlated with negative block (pan6-10)
    pos_ave = row['pand1':'pand5'].mean()
    neg_ave = row['pand6':'pand10'].mean()
    num = np.std([pos_ave, neg_ave])
    affect_check.append(abs(num))
    
    #irb2, irb3, irb4 should be negatively correlated with irb6 and irb7
    #itp1, itp2, itp3 should be negatively correlated with irb6 and irb7
    #dalal1-8 should be negatively correlated with dalal9-dalal16

#context checks
job_only['context_homevsworking'] = context_homevsworking
job_only['context_workvsactivities'] = context_workvsactivities
job_only['context_workvswork'] = context_workvswork
job_only['context_drivevsdrive'] = context_drivevsdrive


job_only['affect_check'] = affect_check

In [ ]:
job_only.head()

In [ ]:
#long string analysis (e.g. max length of same number answered for pf_03:pf_15)
#create features related to long string analysis (feature of how long the string is and feature of what the string consisted of)

max_strings = []
max_answers = []

for index, row in psych_flex_only.iterrows():
    
    groups = groupby(row['pf_03':'pf_15'])
    result = [(label, sum(1 for _ in group)) for label, group in groups]

    max_pair = max(result, key=lambda x:x[1])
    max_string_length = max_pair[1]
    max_answer = max_pair[0]

    max_strings.append(max_string_length)
    
    max_answers.append(max_answer)
    
psych_flex_only['longest_string_count'] = max_strings
psych_flex_only['longest_string_answer'] = max_answers

In [ ]:
#split off completed job and related columns
job_only = data[(data['survey_type'] == 'job')]

print(job_only.shape)
job_only['ParticipantID'].unique().shape

In [ ]:
job_only['dalal16'].value_counts()